# Pointcloud 3D Tree Modelling

This notebook tries to represent a tree in 3D using the deflt repo.

---------------

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import os
import math
import time
import trimesh
import shapely
import pymeshfix
import subprocess
import alphashape
import numpy as np
import open3d as o3d
import logging as log
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial import KDTree
from descartes import PolygonPatch
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from plyfile import PlyData, PlyElement
from skimage.measure import CircleModel, ransac
from sklearn.neighbors import NearestNeighbors
import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.las_utils as las_utils
from misc.quaternion import Quaternion

## Datasets

-------

#### AHN Reconstruction

![AHN Reconstruction](../imgs/ahn_reconstruction.png)

In [ ]:
ahn_tree_path = '../datasets/single_selection/single_121913_487434_AHN.las'
las_utils.read_las(ahn_tree_path)

# TODO: python reconstruction implementation

#### Cyclo Reconstruction

![Cyclo Media Reconstruction](../imgs/cyclo_reconstruction.png)

In [ ]:
cyclo_tree_path = '../datasets/single_selection/single_121913_487434_Cyclo.las'
cyclo_pcd = las_utils.to_o3d(las_utils.read_las(cyclo_tree_path))
cyclo_pcd = cyclo_pcd.voxel_down_sample(voxel_size=0.02)

#### Sonarski Reconstruction

![Sonarski Reconstruction](../imgs/sonarski_reconstruction.png)

In [ ]:
sonarski_tree_path = '../datasets/single_selection/single_121913_487434_Sonarski.las'
las_utils.read_las(sonarski_tree_path)

# TODO: python reconstruction implementation

## Modules

--------

#### 1. Leaf-Wood Filtering

There are 2 filters.
- `surface_variation_filter()`, filter based on the **surface variation** of a point. 
- `curvature_filter()`, filter based on the **curvature** of a point

In [ ]:
def surface_variation_filter(pcd, radius=0.05, threshold=.15):
    pcd.estimate_covariances(
        search_param=o3d.geometry.KDTreeSearchParamRadius(radius=radius))
    eig_val, _ = np.linalg.eig(np.asarray(pcd.covariances))
    eig_val = np.sort(eig_val, axis=1)

    filter_mask = eig_val[:,0] / eig_val.sum(axis=1) < threshold

    return filter_mask

def curvature_filter(pcd, radius, min1=0, max1=100, min2=0, max2=100, min3=0, max3=100):

    # estimate eigenvalues
    pcd.estimate_covariances(
        search_param=o3d.geometry.KDTreeSearchParamRadius(radius=radius))
    eig_val, _ = np.linalg.eig(np.asarray(pcd.covariances))
    eig_val = np.sort(eig_val, axis=1)
    eig_val[eig_val[:,2]==1] = np.zeros(3)
    L1, L2, L3 = eig_val[:,2], eig_val[:,1], eig_val[:,0]

    # filter L1
    filter_L1 = (L1 > L1.min() + (L1.max()-L1.min()) / 100 * min1) & \
        (L1 < L1.min() + (L1.max()-L1.min()) / 100 * max1)

    # filter L2
    filter_L2 = (L2 > L2.min() + (L2.max()-L2.min()) / 100 * min2) & \
        (L2 < L2.min() + (L2.max()-L2.min()) / 100 * max2)

    # filter L3
    filter_L3 = (L3 > L3.min() + (L3.max()-L3.min()) / 100 * min3) & \
        (L3 < L3.min() + (L3.max()-L3.min()) / 100 * max3)

    L1 = (L1 - L1.min()) / ((L1.max()-L1.min()) / 100)
    L2 = (L2 - L2.min()) / ((L2.max()-L2.min()) / 100)
    L3 = (L3 - L3.min()) / ((L3.max()-L3.min()) / 100)


    filter_mask = filter_L1 & filter_L2 & filter_L3
    return filter_mask, (L1,L2,L3)

def filter_leaves(pcd, surface=False):

    pcd_, _ = pcd.remove_radius_outlier(nb_points=4, radius=.05)

    if surface:
        mask = surface_variation_filter(pcd_, .1, .15)
    else:
        mask, l123 = curvature_filter(pcd_, .05, min1=20, min2=35)

    # Visualize result
    wood_pcd = pcd_.select_by_index(np.where(mask)[0])
    wood_pcd = wood_pcd.paint_uniform_color([.5,.3,0])
    leaf_pcd = pcd_.select_by_index(np.where(mask)[0], invert=True)
    leaf_pcd = leaf_pcd.paint_uniform_color([.8,1,.8])
    o3d.visualization.draw_geometries([wood_pcd, leaf_pcd])

    return wood_pcd, l123, pcd_


In [ ]:
pcd = cyclo_pcd
pcd_filtered, l123, pcd_ = filter_leaves(pcd, surface=False)
print(len(pcd.points),'-->', len(pcd_filtered.points))

In [ ]:
import laspy

points = np.asarray(pcd_.points)

las = laspy.create(file_version="1.2", point_format=3)
las.header.offsets = np.min(points, axis=0)
las.x = points[:, 0]
las.y = points[:, 1]
las.z = points[:, 2]

las.add_extra_dim(laspy.ExtraBytesParams(name="L1", type="uint8",description="L1"))
las.add_extra_dim(laspy.ExtraBytesParams(name="L2", type="uint8",description="L2"))
las.add_extra_dim(laspy.ExtraBytesParams(name="L3", type="uint8",description="L3"))
las.L1 = np.asarray(l123[0])
las.L2 = np.asarray(l123[1])
las.L3 = np.asarray(l123[2])
las.write('test.las')

#### 2. 3D-Reconstruction (AdTree)

In [ ]:
# adtree executable
adtree_exe = '../../AdTree-single/build/bin/AdTree.app/Contents/MacOS/AdTree'

In [ ]:
log.basicConfig(
    level=log.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        log.FileHandler("debug.log"),
        log.StreamHandler()
    ]
)

def create_graph(vertices, edges):

    graph = nx.DiGraph()

    for i, vertex in enumerate(vertices):
        graph.add_node(i, x=vertex[0],y=vertex[1],z=vertex[2])

    for i,j in edges:
        graph.add_edge(j, i)

    return graph

def _run_command(command):
    log.info("Command: {}".format(command))
    result = subprocess.run(command, capture_output=True)
    if result.stderr:
        raise subprocess.CalledProcessError(
                returncode = result.returncode,
                cmd = result.args,
                stderr = result.stderr
                )
    return result

def reconstruct_tree(pcd, adtree_exe):
    '''
    Reconstructs the branches of a point coud tree.

    pcd : open3d.geometry.PointCloud
        input tree point cloud

    adtree : str
        path to adtree executable.

    Returns:
    ----------
    PlyData file of the reconstructed input point cloud tree.
    '''

    skeleton_graph = nx.DiGraph()
    vertices, edges = np.array([]), np.array([])

    # create input file system
    tmp_folder = './tmp'
    in_file = os.path.join(tmp_folder, 'tree.xyz')
    out_file = os.path.join(tmp_folder, 'tree_skeleton.ply')
    if not os.path.exists(tmp_folder):
        os.mkdir(tmp_folder)

    try:    
        log.info("Reconstructing...") 
        o3d.io.write_point_cloud(in_file, pcd) # write input file
        result = _run_command([adtree_exe, in_file, out_file]) # run reconstruction
        plydata = PlyData.read(out_file) # read output

        # Convert skeleton to graph
        log.info("Done. Constructing graph...")
        vertices = np.array([[c for c in p] for p in plydata['vertex'].data])
        vertices, reverse_ = np.unique(vertices, axis=0, return_inverse=True)
        edges = np.array([reverse_[edge[0]] for edge in plydata['edge'].data])
        skeleton_graph = create_graph(vertices, edges)
        log.info("Done. Succesful.")
        
    except subprocess.CalledProcessError as e:
        log.error("Failed reconstructing tree:\n{}".format(e.stderr.decode('utf-8')))
    except Exception as e:
        log.error("Failed:\n{}".format(e))

    # clean filesystem
    if os.path.exists(in_file):
        os.remove(in_file)
    if os.path.exists(out_file):
        os.remove(out_file)
    if os.path.isdir(tmp_folder):
        os.rmdir(tmp_folder)

    return skeleton_graph, vertices, edges
    

In [ ]:
skeleton, vertices, edges = reconstruct_tree(pcd_filtered, adtree_exe)
# skeleton, vertices, edges = reconstruct_tree(pcd, adtree_exe)

#### 3. Stem Split

In [ ]:
def plot_skeleton(vertices, edges):
    # visulalize
    colors = [[1, 0, 0] for i in range(len(edges))]
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(vertices)
    line_set.lines = o3d.utility.Vector2iVector(edges)
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # colors = [[0, 0, 0] for i in range(len(edges_sel))]
    # line_set_sel = o3d.geometry.LineSet()
    # line_set_sel.points = o3d.utility.Vector3dVector(vertices)
    # line_set_sel.lines = o3d.utility.Vector2iVector(edges_sel)
    # line_set_sel.colors = o3d.utility.Vector3dVector(colors)

    v_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(vertices))
    v_pcd = v_pcd.paint_uniform_color([0,0.5,0])
    o3d.visualization.draw_geometries([line_set, v_pcd])

def branch_path(graph, start):

    path = [start]
    while graph.out_degree(path[-1]) == 1:
        for node in graph.successors(path[-1]):
            path.append(node)
            break

    print(path[0], '-->', path[-1], '(nodes:',len(path),')')
    return path

def get_stem(skeleton, pcd):
    '''
    Returns the stem of a point cloud given the skeleton. 
    The Stem is the part of the tree till the first branch split
    
    Params
    -----------
    skeleton : NetworkX Graph
        The tree skeleton
    pcd : o3d.geometry.PointCloud
        The point cloud of the tree

    Returns
    -----------
    A open3d point cloud of the stem of the given tree
    '''

    # get start node
    zs = nx.get_node_attributes(skeleton, 'z')
    start_node = min(zs, key=zs.get)

    # get path till first split
    stem_path = branch_path(skeleton, start_node)
    stem_points = np.array([list(skeleton.nodes[node].values()) for node in stem_path])

    # Create tree points
    tree_points = np.array(pcd.points)
    mask_idx = np.where(tree_points[:,2] < stem_points[:,2].max())[0]
    tree = KDTree(tree_points[mask_idx])

    # Filter tree points
    selection = set()
    num_ = int(np.linalg.norm(stem_points[1]-stem_points[0]) / 0.05)
    stem_points = np.linspace(start=stem_points[0], stop=stem_points[1], num=num_)
    for result in tree.query_ball_point(stem_points, .75):
        selection.update(result) 
    selection = mask_idx[list(selection)]

    # Visualize
    stem_pcd = pcd.select_by_index(selection)
    stem_pcd = stem_pcd.paint_uniform_color([.4,.7,0])
    crown_pcd = pcd.select_by_index(selection, invert=True)
    o3d.visualization.draw_geometries([stem_pcd, crown_pcd])

    return stem_pcd, crown_pcd


In [ ]:
stem_pcd, crown_pcd = get_stem(skeleton, pcd)

In [ ]:
plot_skeleton(vertices, edges)

#### 4. Stem Analysis

In [ ]:
def simplify_mesh(mesh, num_triangles):
    return mesh.simplify_quadric_decimation(target_number_of_triangles=num_triangles)

def show_mesh(m, color=[0,0,0]):
    m.paint_uniform_color(color)
    m.compute_vertex_normals()
    o3d.visualization.draw_geometries([m])


In [ ]:
def plot_stem_rims(rim_array, resolution=15, pcd=None):
    
    geometries = []
    for result in rim_array:
        c = result[:3]
        r = result[6]

        line_points = np.zeros((0,3))
        for i in range(resolution):
            phi = i*np.pi/(resolution/2)
            line_points = np.vstack((line_points, (c[0] + r*np.cos(phi), c[1] + r*np.sin(phi), c[2])))

        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(line_points)
        line_set.lines = o3d.utility.Vector2iVector([(i,i+1) for i in range(resolution-1)])
        line_set.colors = o3d.utility.Vector3dVector(np.zeros((resolution-1,3)))
        geometries.append(line_set)

    if pcd is not None:
        geometries.append(pcd)

    o3d.visualization.draw_geometries(geometries)

def rodrigues_rot(points, vector1, vector2):
    """RODRIGUES ROTATION
    - Rotate given points based on a starting and ending vector
    - Axis k and angle of rotation theta given by vectors n0,n1
    P_rot = P*cos(theta) + (k x P)*sin(theta) + k*<k,P>*(1-cos(theta))"""

    if points.ndim == 1:
        points = points[np.newaxis, :]

    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)
    k = np.cross(vector1, vector2)
    if np.sum(k) != 0:
        k = k / np.linalg.norm(k)
    theta = np.arccos(np.dot(vector1, vector2))

    # MATRIX MULTIPLICATION
    P_rot = np.zeros((len(points), 3))
    for i in range(len(points)):
        P_rot[i] = (
            points[i] * np.cos(theta)
            + np.cross(k, points[i]) * np.sin(theta)
            + k * np.dot(k, points[i]) * (1 - np.cos(theta))
        )
    return P_rot

def circumferential_completeness_index(fitted_circle_centre, estimated_radius, slice_points):
    """
    Computes the Circumferential Completeness Index (CCI) of a fitted circle.
    Args:
        fitted_circle_centre: x, y coords of the circle centre
        estimated_radius: circle radius
        slice_points: the points the circle was fitted to
    Returns:
        CCI
    """

    sector_angle = 4.5  # degrees
    num_sections = int(np.ceil(360 / sector_angle))
    sectors = np.linspace(-180, 180, num=num_sections, endpoint=False)

    centre_vectors = slice_points[:, :2] - fitted_circle_centre
    norms = np.linalg.norm(centre_vectors, axis=1)

    centre_vectors = centre_vectors / np.atleast_2d(norms).T
    centre_vectors = centre_vectors[
        np.logical_and(norms >= 0.8 * estimated_radius, norms <= 1.2 * estimated_radius)
    ]

    sector_vectors = np.vstack((np.cos(sectors), np.sin(sectors))).T
    CCI = (
        np.sum(
            [
                np.any(
                    np.degrees(
                        np.arccos(
                            np.clip(np.einsum("ij,ij->i", np.atleast_2d(sector_vector), centre_vectors), -1, 1)
                        )
                    )
                    < sector_angle / 2
                )
                for sector_vector in sector_vectors
            ]
        )
        / num_sections
    )

    return CCI

def fit_circle_2D(points):

    CCI = 0
    r = 0
    xc, yc = np.mean(points[:, :2], axis=0)

    # Fit circle in new 2D coords with RANSAC
    if points.shape[0] >= 20:

        model_robust, inliers = ransac(
            points[:, :2],
            CircleModel,
            min_samples=int(points.shape[0] * 0.3),
            residual_threshold=0.05,
            max_trials=1000,
        )
        xc, yc = model_robust.params[0:2]
        r = model_robust.params[2]
        CCI = circumferential_completeness_index([xc, yc], r, points[:, :2])

    if CCI < 0.3:
        r = 0
        xc, yc = np.mean(points[:, :2], axis=0)
        CCI = 0

    return CCI, r, xc, yc

def fit_circle_3D(points, V):
    """
    Fits a circle using Random Sample Consensus (RANSAC) to a set of points in a plane perpendicular to vector V.
    Args:
        points: Set of points to fit a circle to using RANSAC.
        V: Axial vector of the cylinder you're fitting.
    Returns:
        cyl_output: numpy array of the format [[x, y, z, x_norm, y_norm, z_norm, radius, CCI, 0, 0, 0, 0, 0, 0]]
    """

    P = points[:, :3]
    P_mean = np.mean(P, axis=0)
    P_centered = P - P_mean
    normal = V / np.linalg.norm(V)
    if normal[2] < 0:  # if normal vector is pointing down, flip it around the other way.
        normal = normal * -1

    # Project points to coords X-Y in 2D plane
    P_xy = rodrigues_rot(P_centered, normal, [0, 0, 1])

    CCI, r, xc, yc = fit_circle_2D(P_xy)

    # Transform circle center back to 3D coords
    cyl_centre = rodrigues_rot(np.array([[xc, yc, 0]]), [0, 0, 1], normal) + P_mean
    cyl_output = np.array(
        [
            [
                cyl_centre[0, 0],
                cyl_centre[0, 1],
                cyl_centre[0, 2],
                normal[0],
                normal[1],
                normal[2],
                r,
                CCI
            ]
        ]
    )
    return cyl_output

def fit_cylinder(pcd, skeleton_points, num_neighbours, plot=False):
    """
    Fits a 3D line to the skeleton points cluster provided.
    Uses this line as the major axis/axial vector of the cylinder to be fitted.
    Fits a series of circles perpendicular to this axis to the point cloud of this particular stem segment.
    Args:
        pcd: The cluster of points belonging to the segment of the branch.
        skeleton_points: A single cluster of skeleton points which should represent a segment of a tree/branch.
        num_neighbours: The number of skeleton points to use for fitting each circle in the segment. lower numbers
                        have fewer points to fit a circle to, but higher numbers are negatively affected by curved
                        branches. Recommend leaving this as it is.
    Returns:
        cyl_array: a numpy array based representation of the fitted circles/cylinders.
    """

    point_cloud = np.array(pcd.points)[:, :3]
    skeleton_points = skeleton_points[:, :3]
    cyl_array = np.zeros((0, 8))
    line_centre = np.mean(skeleton_points[:, :3], axis=0)
    _, _, vh = np.linalg.svd(line_centre - skeleton_points)
    line_v_hat = vh[0] / np.linalg.norm(vh[0])

    if skeleton_points.shape[0] <= num_neighbours:
        group = skeleton_points
        line_centre = np.mean([np.min(group[:, :3], axis=0), np.max(group[:, :3], axis=0)], axis=0)
        length = np.linalg.norm(np.max(group, axis=0) - np.min(group, axis=0))
        plane_slice = point_cloud[
            np.linalg.norm(abs(line_v_hat * (point_cloud - line_centre)), axis=1) < (length / 2)
        ]  # calculate distances to plane at centre of line.
        if plane_slice.shape[0] > 0:
            cylinder = fit_circle_3D(plane_slice, line_v_hat)
            cyl_array = np.vstack((cyl_array, cylinder))
    else:
        for i in tqdm(range(skeleton_points.shape[0])):
            if skeleton_points.shape[0] > num_neighbours:
                nn = NearestNeighbors()
                nn.fit(skeleton_points)
                starting_point = np.atleast_2d(skeleton_points[np.argmin(skeleton_points[:, 2])])
                group = skeleton_points[nn.kneighbors(starting_point, n_neighbors=num_neighbours)[1][0]]
                line_centre = np.mean(group[:, :3], axis=0)
                length = np.linalg.norm(np.max(group, axis=0) - np.min(group, axis=0))
                plane_slice = point_cloud[
                    np.linalg.norm(abs(line_v_hat * (point_cloud - line_centre)), axis=1) < (length / 2)
                ]  # calculate distances to plane at centre of line.
                if plane_slice.shape[0] > 0:
                    cylinder = fit_circle_3D(plane_slice, line_v_hat)
                    cyl_array = np.vstack((cyl_array, cylinder))
                skeleton_points = np.delete(skeleton_points, np.argmin(skeleton_points[:, 2]), axis=0)

    if plot:
        plot_stem_rims(cyl_array, pcd=pcd)
    
    return cyl_array
    

In [ ]:
def stem_skeleton(pcd, slice_thickness=0.15):

    points = np.asarray(pcd.points)
    min_z, max_z = min(points[:, 2])-slice_thickness, max(points[:, 2])+slice_thickness
    bins = np.arange(min_z, max_z, slice_thickness)
    slice_ind = np.digitize(points[:,2], bins, right=True)

    skeleton_points = np.zeros((0, 3))
    for i in np.unique(slice_ind):
        new_slice = points[slice_ind==i]
        if new_slice.shape[0] > 0:
            median = np.median(new_slice, axis=0)
            skeleton_points = np.vstack((skeleton_points, median))

    return skeleton_points
    

In [ ]:
skeleton_points = stem_skeleton(stem_pcd, .3)
skeleton_points.shape

In [ ]:
rim_arr = fit_cylinder(stem_pcd, skeleton_points, 3, True)

##### Generate meshes

In [ ]:
def stem_to_cyl_mesh(rim_array):
    stem_cyl = trimesh.creation.cylinder(radius=np.mean(rim_array[:,6]), sections=20, segment=rim_array[[0,-1],:3]).as_open3d
    stem_cyl.paint_uniform_color([0.348, 0.11, 0])
    stem_cyl.compute_vertex_normals()
    return stem_cyl
    
def stem_to_mesh(rim_array, resolution=15):

    circle_fits = [(rim[:3], rim[6]) for rim in rim_arr]
    num_slices = len(circle_fits)

    # compute directional axis
    Z = np.array([0, 0, 1], dtype=float)
    centers = np.array([c for c, r in circle_fits])
    axis = centers[-1] - centers[0]
    l = np.linalg.norm(axis)
    if l <= 1e-12:
        axis=Z

    rot = Quaternion.fromData(Z, axis).to_matrix()

    # create vertices
    angles = [2*math.pi*i/float(resolution) for i in range(resolution)]
    rim = np.array([[math.cos(theta), math.sin(theta), 0.0]
        for theta in angles])
    rim = np.dot(rot, rim.T).T

    rims = np.array([
        rim * r + c
        for c, r in circle_fits], dtype=float)
    rims = rims.reshape((-1,3))

    vertices = np.vstack([[centers[0], centers[-1]], rims ])

    # create faces
    bottom_fan = np.array([
        [0, (i+1)%resolution+2, i+2]
        for i in range(resolution) ], dtype=int)

    top_fan = np.array([
        [1, i+2+resolution*(num_slices-1), (i+1)%resolution+2+resolution*(num_slices-1)]
        for i in range(resolution) ], dtype=int)

    slice_fan = np.array([
        [[2+i, (i+1)%resolution+2, i+resolution+2],
            [i+resolution+2, (i+1)%resolution+2, (i+1)%resolution+resolution+2]]
        for i in range(resolution) ], dtype=int)
    slice_fan = slice_fan.reshape((-1, 3), order="C")

    side_fan = np.array([
        slice_fan + resolution*i 
        for i in range(num_slices-1)], dtype=int)
    side_fan = side_fan.reshape((-1, 3), order="C") 

    faces = np.vstack([bottom_fan, top_fan, side_fan])

    # create mesh
    mesh = trimesh.base.Trimesh(vertices, faces).as_open3d

    return mesh


In [ ]:
stem_cyl_mesh = stem_to_cyl_mesh(rim_arr)
show_mesh(stem_cyl_mesh, [0.348, 0.11, 0])

In [ ]:
stem_mesh = stem_to_mesh(rim_arr)
show_mesh(stem_mesh, [0.348, 0.11, 0])

#### 5. Crown Analysis

In [ ]:
def pcd_convex_hull(pcd, plot=True):
    log.info(f'Convex Hull for pcd with {len(pcd.points)} points.')
    o3d_mesh, _ = pcd.compute_convex_hull()
    o3d_mesh.paint_uniform_color([0,0.78,0])
    o3d_mesh.compute_vertex_normals()
    log.info(f'Done. Crown volume: {o3d_mesh.get_volume():.2f}m3')
    
    if plot: # Visualize
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.plot_trisurf(*zip(*o3d_mesh.vertices), triangles=o3d_mesh.triangles, color='green')
        plt.show()

    return o3d_mesh

def alpha_shape(pcd, alpha=.8, plot=True):
    log.info(f'Alpha Shapes for pcd with {len(pcd.points)} points.')
    start = time.time()
    pcd_pts = np.asarray(pcd.points)
    mesh = alphashape.alphashape(pcd_pts, alpha)
    log.info(f'Done. {time.time()-start:.2f}s.')
    
    # Repair
    log.info(f'Repair broken faces...')
    clean_points, clean_faces = pymeshfix.clean_from_arrays(mesh.vertices,  mesh.faces)
    mesh = trimesh.base.Trimesh(clean_points, clean_faces)
    mesh.fix_normals()
    log.info(f'Done. Crown volume: {mesh.volume:.2f}m3')
    
    if plot: # Visualize
        fig = plt.figure()
        ax = plt.axes(projection='3d')
        ax.plot_trisurf(*zip(*mesh.vertices), triangles=mesh.faces, color='green')
        plt.show()

    # convert
    o3d_mesh = mesh.as_open3d
    o3d_mesh.compute_vertex_normals()
    o3d_mesh.paint_uniform_color([0,0.78,0])

    return o3d_mesh

def plot_meshlines(pcds, mesh):
    mesh_lines = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
    mesh_lines.paint_uniform_color((1, 0, 0))
    geometries = pcds + [mesh_lines]
    o3d.visualization.draw_geometries(geometries)


In [ ]:
voxel_size = 0.3

# Down sample crown
crown_sampled = crown_pcd.voxel_down_sample(voxel_size)

In [ ]:
mesh_ch = pcd_convex_hull(crown_sampled)

In [ ]:
mesh_as = alpha_shape(crown_sampled)

In [ ]:
plot_meshlines([stem_pcd, crown_pcd], mesh)

Top-down hull

In [ ]:
def grid_project(pcd, voxel_size):
    pts = np.array(pcd.points)
    pts[:,2] = 0
    pcd_ = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts))
    pcd_ = pcd_.voxel_down_sample(voxel_size)
    pts = np.asarray(pcd_.points)[:,:2]
    return pts

In [ ]:
# Stem shape
proj_pts = grid_project(stem_pcd, 0.02)
stem_shape = alphashape.alphashape(proj_pts, 0.5)
stem_shape = stem_shape.buffer(0.01)
tree_center = np.hstack(stem_shape.centroid.coords)

# Crown shape
proj_pts = grid_project(crown_pcd, 0.3)
crown_shape = alphashape.alphashape(proj_pts, 2)
crown_shape = crown_shape.buffer(0.1)

# Center
proj_pts_shifted = proj_pts-[tree_center]
stem_shape_shifted = shapely.affinity.translate(stem_shape, *(-tree_center))
crown_shape_shifted = shapely.affinity.translate(crown_shape, *(-tree_center))

# Initialize plot
fig, ax = plt.subplots()
ax.plot(0, marker='x', c='k', markersize=5)
ax.add_patch(PolygonPatch(crown_shape_shifted, alpha=.15, color='green', label='Crown'))
ax.add_patch(PolygonPatch(stem_shape_shifted, alpha=.8, color='brown', label='Stem'))
lim = np.max(np.abs(proj_pts_shifted))
ax.set_xlim(-lim-.5,lim+.5)
ax.set_ylim(-lim-.5,lim+.5)
ax.legend()
ax.set_title('Tree Projection')
plt.show()

## Overig

---------

In [ ]:
# Stam doet gek...

In [ ]:
skim = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(vertices[leaf_nodes]))
o3d.visualization.draw_geometries([skim])

In [ ]:
skeleton_sel = skeleton.subgraph([node for node, n in skeleton.out_degree() if n != 0])
edges_sel = np.array([(i,j) for i,j in skeleton_sel.edges])
plot_skeleton(vertices, edges, edges_sel)